In [92]:
!pip install -U transformers
!pip install -q transformers datasets
!pip install -q git+https://github.com/huggingface/peft
import pandas as pd
import torch
import datasets
from datasets import load_dataset
from datasets import Dataset, DatasetDict, concatenate_datasets
import numpy as np

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [93]:
train_test_ratio = 0.2

In [94]:
from sklearn.model_selection import train_test_split

dataset = pd.read_csv('/content/en-annotated.tsv', sep='\t', header=None)
print(dataset)

                                                       0        1
0                                                  , ...        1
1                                                      !  1, 4, 7
2      ... And I don't think we need to discuss the T...     8, 1
3                            * So get up out of your bed        1
4      A confession that you hired [PERSON] ... and a...     1, 6
...                                                  ...      ...
17523                   Your opinion might be valuable .        8
17524                                      Your orders .        8
17525              Your ship's been in lots of battles .        8
17526                         Your wine , your Majesty .        8
17527                                            Yours ?        8

[17528 rows x 2 columns]


In [95]:
train, test = train_test_split(dataset, test_size=train_test_ratio)

train = train.rename(columns={0: 'dialogue', 1: 'summary'})
test = test.rename(columns={0: 'dialogue', 1: 'summary'})

print(f"Train dataset size: {len(train)}")
print(f"Test dataset size: {len(test)}")

print("Sample example:")
print(train.iloc[0])

Train dataset size: 14022
Test dataset size: 3506
Sample example:
dialogue    If the rights of the individual are not respec...
summary                                                     1
Name: 6223, dtype: object


In [96]:
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(test)

ds = DatasetDict()

ds['train'] = tds
ds['test'] = vds

dataset = ds
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['dialogue', 'summary', '__index_level_0__'],
        num_rows: 14022
    })
    test: Dataset({
        features: ['dialogue', 'summary', '__index_level_0__'],
        num_rows: 3506
    })
})


In [97]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_id="google/mt5-small"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [98]:
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["dialogue"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
input_lenghts = [len(x) for x in tokenized_inputs["input_ids"]]

max_source_length = int(np.percentile(input_lenghts, 100)) #can be 85
print(f"Max source length: {max_source_length}")

tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
target_lenghts = [len(x) for x in tokenized_targets["input_ids"]]

max_target_length = int(np.percentile(target_lenghts, 100)) #can be 90
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/17528 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Max source length: 89


Map:   0%|          | 0/17528 [00:00<?, ? examples/s]

Max target length: 8


In [99]:
def preprocess_function(sample,padding="max_length"):
    inputs = ["classify emotions: " + item for item in sample["dialogue"]]

    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)
    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

tokenized_dataset["train"].save_to_disk("data/train")
tokenized_dataset["test"].save_to_disk("data/eval")

Map:   0%|          | 0/14022 [00:00<?, ? examples/s]

Map:   0%|          | 0/3506 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels']


Saving the dataset (0/1 shards):   0%|          | 0/14022 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3506 [00:00<?, ? examples/s]

In [100]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType

model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")

In [101]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 688,128 || all params: 300,864,896 || trainable%: 0.2287


In [102]:
from transformers import DataCollatorForSeq2Seq

label_pad_token_id = -100

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [103]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

output_dir="tutorial"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
	auto_find_batch_size=True,
    learning_rate=1e-3, # higher learning rate
    num_train_epochs=4,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=250,
    save_strategy="no",
    report_to="tensorboard",
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
)
# model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [105]:
trainer.train()

Step,Training Loss
250,8.286800
500,1.772900
750,1.613300
1000,1.586900
1250,1.536500
1500,1.531700
1750,1.484700
2000,1.495400
2250,1.511500
2500,1.482900


TrainOutput(global_step=7012, training_loss=1.7378098375785032, metrics={'train_runtime': 726.2299, 'train_samples_per_second': 77.232, 'train_steps_per_second': 9.655, 'total_flos': 5608308714504192.0, 'train_loss': 1.7378098375785032, 'epoch': 4.0})

In [106]:
peft_model_id=output_dir
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

('tutorial/tokenizer_config.json',
 'tutorial/special_tokens_map.json',
 'tutorial/spiece.model',
 'tutorial/added_tokens.json',
 'tutorial/tokenizer.json')

In [107]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc.
peft_model_id = output_dir
config = PeftConfig.from_pretrained(peft_model_id)

# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model.eval()

print("Peft model loaded")

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Peft model loaded


In [167]:
# use the first sample of the test set
sample = dataset['test'][42]

input_ids = tokenizer(sample["dialogue"], return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=40, do_sample=False, top_p=0.9)
print(f"input sentence: {sample['dialogue']}\n{'---'* 20}")

print(f"summary:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")

input sentence: [PERSON] , you're asking me to make a total commitment of men and money on the basis of a hunch .
------------------------------------------------------------
summary:
1
